In [1]:
import os
# 상위 폴더로 이동
os.chdir('..')

In [2]:
from data_presets import ClassificationPresetEval
from torchvision.transforms.functional import InterpolationMode
from datasets import load_from_disk
from transformers import AutoModelForImageClassification
from image_utils import run_model, get_metric_from_df, plot_rocauc_from_df, plot_cm_from_df, create_hook, get_embedding_layer
import glob

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
folder_paths = [
    "./cifar100_facebook_convnext-small-224",
]

In [4]:
hf_dirs = []
for folder in folder_paths:
    hf_dirs.extend(glob.glob(f"{folder}/*/best.hf"))

In [5]:
hf_dirs

['./cifar100_facebook_convnext-small-224/convnext-small-224_20241021_0247/best.hf',
 './cifar100_facebook_convnext-small-224/convnext-small-224_20241021_0327/best.hf',
 './cifar100_facebook_convnext-small-224/convnext-small-224_20241021_0444/best.hf',
 './cifar100_facebook_convnext-small-224/convnext-small-224_merge/best.hf']

In [21]:
import glob
from safetensors import safe_open
from safetensors.torch import save_file
import shutil
import os
import torch

# 모델 파일 경로 리스트 생성
hf_dirs = []
for folder in folder_paths:
    hf_dirs.extend(glob.glob(f"{folder}/*/best.hf/model.safetensors"))

# 합산된 파라미터를 저장할 딕셔너리 초기화
summed_params = {}

# 모델의 수 계산
num_models = len(hf_dirs)
print(f"총 {num_models}개의 모델을 평균합니다.")

# 각 모델 파일에서 파라미터 로드 및 합산
for idx, path in enumerate(hf_dirs):
    print(f"모델 로딩 중 ({idx+1}/{num_models}): {path}")
    params = {}
    with safe_open(path, framework="pt", device="cpu") as f:
        for key in f.keys():
            params[key] = f.get_tensor(key)
    if idx == 0:
        # 첫 번째 모델의 파라미터로 초기화
        for key in params.keys():
            summed_params[key] = params[key].clone()
    else:
        # 이후 모델의 파라미터를 합산
        for key in params.keys():
            summed_params[key] += params[key]

# 합산된 파라미터를 모델 수로 나누어 평균 계산
for key in summed_params.keys():
    summed_params[key] /= num_models

# 첫 번째 모델의 'best.hf' 폴더를 복사하여 새로운 폴더 생성
best_hf_dir = os.path.dirname(hf_dirs[0])
dest_folder = 'cifar100_facebook_convnext-small-224/best.hf'

# 기존 폴더가 있으면 삭제
if os.path.exists(dest_folder):
    shutil.rmtree(dest_folder)

# 폴더 복사
shutil.copytree(best_hf_dir, dest_folder)

# 평균된 파라미터를 새로운 safetensors 파일로 저장
save_file(
    summed_params,
    os.path.join(dest_folder, 'model.safetensors'),
    metadata={"format": "pt"}  # 메타데이터 추가
)
print(f"새로운 모델이 '{os.path.join(dest_folder, 'model.safetensors')}'로 저장되었습니다.")


총 3개의 모델을 평균합니다.
모델 로딩 중 (1/3): ./cifar100_facebook_convnext-small-224/convnext-small-224_20241021_0247/best.hf/model.safetensors
모델 로딩 중 (2/3): ./cifar100_facebook_convnext-small-224/convnext-small-224_20241021_0327/best.hf/model.safetensors
모델 로딩 중 (3/3): ./cifar100_facebook_convnext-small-224/convnext-small-224_20241021_0444/best.hf/model.safetensors
새로운 모델이 'cifar100_facebook_convnext-small-224/best.hf/model.safetensors'로 저장되었습니다.


In [24]:
# config parameter 설정
config = {
    "model_dir" : '/workspace/model/hugginface_image_classifcation/cifar100_facebook_convnext-small-224/best.hf',
    "dataset_dir" : '/workspace/model/hugginface_image_classifcation/data/hfdataset/test',
}

In [26]:
dataset = load_from_disk(config['dataset_dir'])
labels = dataset['test'].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
    
model = AutoModelForImageClassification.from_pretrained(
        config['model_dir'],
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes = True,
    )

def preprocess_valid(batch, valid_transforms):
    """Apply train_transforms across a batch."""
    if "image" in batch:
        batch["pixel_values"] = [
            valid_transforms(image.convert("RGB")) for image in batch["image"]
    ]
    return batch

valid_transform = ClassificationPresetEval(
            crop_size=224,
            resize_size=256,
            interpolation=InterpolationMode.BILINEAR,
            use_v2=True,
        )

valid_ds = dataset['test']
valid_ds.set_transform(lambda batch: preprocess_valid(batch, valid_transform))

In [27]:
valid_df = run_model(model, valid_ds, id2label)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [28]:
get_metric_from_df(valid_df)

{'accuracy': 0.873,
 'top2_accuracy': 0.9491,
 'precision': 0.873,
 'recall': 0.873,
 'f1_score': 0.873,
 'roc_auc': 0.9987518367676768,
 'specificity': 0.9987171717171717}